##**Assignment 2**
##Задача 4

Алгоритм состоит из следующих этапов:

массив разбивается на подмассивы, каждый из которых сортируется отдельно на GPU;

далее выполняются параллельные проходы слияния;

измеряется время выполнения для массивов размером 10 000 и 100 000 элементов.

In [2]:
!nvidia-smi

Tue Dec 30 05:17:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
%%writefile gpu_merge_sort.cu
#include <iostream>                 // cout/cin для вывода в консоль
#include <vector>                   // vector<int> для хранения массива на CPU
#include <algorithm>                // sort() для эталонной CPU-сортировки
#include <random>                   // mt19937 и распределение для генерации случайных чисел
#include <chrono>                   // измерение времени на CPU
#include <climits>                  // INT_MAX / INT_MIN для паддинга и граничных значений
#include <cuda_runtime.h>           // CUDA runtime: cudaMalloc/cudaMemcpy/cudaEvent и т.д.

using namespace std;                // чтобы не писать std:: везде

static const int CHUNK = 1024;      // размер "куска" (chunk), который сортирует один CUDA-блок; степень 2 нужна для bitonic sort
static const int THREADS = 256;     // количество потоков в блоке для merge kernel

// Макрос: проверка ошибок CUDA-вызовов (если ошибка - печатаем и завершаем программу)
#define CUDA_CHECK(x) do{ cudaError_t e=(x); if(e!=cudaSuccess){ \
  printf("CUDA error %s:%d: %s\n",__FILE__,__LINE__,cudaGetErrorString(e)); exit(1);} }while(0)

// Сортировка чанков (bitonic внутри блока)

// kernel: каждый CUDA-блок сортирует свой подмассив длины CHUNK, используя shared memory (быстро, потому что shared быстрее global)
__global__ void bitonicSortChunks(int* d, int N) {                    // d - массив в глобальной памяти GPU, N - реальный размер массива
    __shared__ int s[CHUNK];                                          // shared memory: общий буфер для потоков текущего блока (вмещает CHUNK чисел)
    int base = blockIdx.x * CHUNK;                                    // base - начало чанка, который обрабатывает этот блок

    for (int i = threadIdx.x; i < CHUNK; i += blockDim.x) {           // каждый поток грузит несколько элементов чанка (шаг = blockDim.x)
        int idx = base + i;                                           // idx - глобальный индекс элемента в массиве d
        s[i] = (idx < N) ? d[idx] : INT_MAX;                          // если вышли за границы - кладём INT_MAX (паддинг), чтобы сортировка не ломалась
    }
    __syncthreads();                                                  // синхронизация: все данные должны оказаться в shared перед сортировкой

    for (int k = 2; k <= CHUNK; k <<= 1) {                            // k - размер bitonic-последовательности (растёт: 2,4,8,...CHUNK)
        for (int j = k >> 1; j > 0; j >>= 1) {                        // j - “дистанция” для сравнения пар (уменьшается внутри стадии)
            for (int i = threadIdx.x; i < CHUNK; i += blockDim.x) {   // каждый поток обрабатывает свой набор i (часть элементов)
                int ixj = i ^ j;                                      // ixj - индекс “пары” для i
                if (ixj > i) {                                        // сравниваем пару только один раз, чтобы не дублировать работу
                    bool up = ((i & k) == 0);                         // up=true: сортируем по возрастанию, иначе по убыванию (зависит от стадии k)
                    int a = s[i], b = s[ixj];                         // читаем два значения из shared, которые нужно сравнить
                    if ((up && a > b) || (!up && a < b)) {            // если порядок неверный для выбранного направления
                        s[i] = b;                                     // меняем местами (swap вручную, чтобы не тянуть std::swap на device)
                        s[ixj] = a;                                   // вторая часть обмена
                    }
                }
            }
            __syncthreads();                                          // важная синхронизация после каждого шага - иначе потоки будут мешать друг другу
        }
    }

    for (int i = threadIdx.x; i < CHUNK; i += blockDim.x) {           // после сортировки записываем результат обратно в global memory
        int idx = base + i;                                           // глобальный индекс для записи
        if (idx < N) d[idx] = s[i];                                   // пишем только реальные элементы (без INT_MAX паддинга)
    }
}

// Параллельное слияние (merge-path)

// device-функция merge-path: находит, сколько элементов нужно взять из A для позиции k в объединённом массиве
// Мы хотим out[k]. Пусть i элементов берём из A, тогда j=k-i берём из B. Ищем i бинарным поиском.
__device__ __forceinline__ int mergePath(const int* A,int m,const int* B,int n,int k){
    int lo = max(0, k - n);                                           // минимально возможное i (если взять слишком много из B, выйдем за границы)
    int hi = min(k, m);                                               // максимально возможное i (нельзя взять больше m элементов из A)
    while (lo < hi) {                                                 // бинарный поиск по i
        int i = (lo + hi) >> 1;                                       // пробуем i примерно посередине
        int j = k - i;                                                // тогда из B берём j

        int Ai   = (i < m) ? A[i] : INT_MAX;                          // A[i] или +∞ если i==m (за правой границей)
        int Aim1 = (i > 0) ? A[i-1] : INT_MIN;                        // A[i-1] или -∞ если i==0 (за левой границей)
        int Bj   = (j < n) ? B[j] : INT_MAX;                          // B[j] или +∞ если j==n
        int Bjm1 = (j > 0) ? B[j-1] : INT_MIN;                        // B[j-1] или -∞ если j==0

        if (Ai < Bjm1) lo = i + 1;                                    // Ai слишком маленький → взяли мало из A → увеличиваем i
        else if (Aim1 > Bj) hi = i;                                   // Aim1 слишком большой → взяли много из A → уменьшаем i
        else return i;                                                // нашли корректную точку разбиения (i подходит)
    }
    return lo;                                                        // возвращаем найденное i
}

// kernel merge: сливает пары сегментов длины width; каждый поток пишет один элемент результата (позицию k)
__global__ void mergePass(const int* in, int* out, int N, int width) {
    int pairId = blockIdx.x;                                          // номер пары сегментов (по оси x): 0,1,2...
    int base = pairId * (2 * width);                                  // начало пары: [base ... base+2*width)

    int m = max(0, min(width, N - base));                             // реальная длина левого сегмента (учитываем, что в конце может быть меньше width)
    int n = max(0, min(width, N - (base + width)));                   // реальная длина правого сегмента
    int outLen = m + n;                                               // сколько всего элементов будет после слияния (левая+правая часть)

    int k = blockIdx.y * blockDim.x + threadIdx.x;                    // позиция k в объединённом сегменте, которую считает данный поток
    if (k >= outLen) return;                                          // если поток “вылез” за длину - он ничего не делает

    const int* A = in + base;                                         // указатель на начало левого сегмента во входном массиве
    const int* B = in + base + width;                                 // указатель на начало правого сегмента

    int i = mergePath(A, m, B, n, k);                                 // сколько элементов нужно “взять” из A до позиции k
    int j = k - i;                                                    // сколько элементов взято из B

    int a = (i < m) ? A[i] : INT_MAX;                                 // кандидат из A для позиции k
    int b = (j < n) ? B[j] : INT_MAX;                                 // кандидат из B для позиции k

    out[base + k] = (a <= b) ? a : b;                                 // выбираем меньший кандидат и записываем в выход
}

// Хост-функция: GPU MERGE SORT

// Функция на CPU: выделяет память на GPU, запускает kernels и возвращает результат обратно на CPU
static void gpuMergeSort(vector<int>& h) {
    int N = (int)h.size();                                            // размер массива на CPU

    int *dA=nullptr, *dB=nullptr;                                     // два буфера на GPU: один для чтения, второй для записи (ping-pong)
    CUDA_CHECK(cudaMalloc(&dA, N * sizeof(int)));                     // выделяем память под dA на GPU
    CUDA_CHECK(cudaMalloc(&dB, N * sizeof(int)));                     // выделяем память под dB на GPU
    CUDA_CHECK(cudaMemcpy(dA, h.data(), N * sizeof(int), cudaMemcpyHostToDevice)); // копируем данные CPU → GPU в dA

    int numChunks = (N + CHUNK - 1) / CHUNK;                          // сколько блоков нужно, чтобы покрыть N элементов чанками по CHUNK
    bitonicSortChunks<<<numChunks, 256>>>(dA, N);                     // сортируем каждый чанк отдельно (получаем много маленьких отсортированных сегментов)
    CUDA_CHECK(cudaGetLastError());                                   // проверяем, не было ли ошибки запуска kernel
    CUDA_CHECK(cudaDeviceSynchronize());                              // ждём, пока kernel полностью завершится

    int width = CHUNK;                                                // текущая длина отсортированного сегмента (сначала = CHUNK)
    bool ping = true;                                                 // ping=true: in=dA out=dB, потом меняем местами

    while (width < N) {                                               // продолжаем, пока сегменты не станут размером хотя бы N
        int numPairs = (N + (2 * width) - 1) / (2 * width);           // сколько пар сегментов нужно слить (каждая пара даёт новый сегмент 2*width)

        int maxOutLen = min(2 * width, N);                            // максимальная длина результата в одной паре (обычно 2*width, но N может быть меньше)
        int blocksY = (maxOutLen + THREADS - 1) / THREADS;            // сколько блоков по оси y нужно, чтобы покрыть все k (один поток = один k)

        dim3 block(THREADS);                                          // конфигурация: THREADS потоков в одном блоке
        dim3 grid(numPairs, blocksY);                                 // grid.x - пары сегментов, grid.y - “слои” по k внутри пары

        const int* in = ping ? dA : dB;                               // выбираем, откуда читаем (вход)
        int* out = ping ? dB : dA;                                    // выбираем, куда пишем (выход)

        mergePass<<<grid, block>>>(in, out, N, width);                // выполняем один проход слияния для всех пар сегментов
        CUDA_CHECK(cudaGetLastError());                               // проверяем запуск merge kernel
        CUDA_CHECK(cudaDeviceSynchronize());                          // ждём завершения merge kernel

        ping = !ping;                                                 // меняем буферы местами на следующий проход
        width *= 2;                                                   // после merge сегменты удваиваются
    }

    int* dRes = ping ? dA : dB;                                       // выбираем, в каком буфере лежит конечный результат
    CUDA_CHECK(cudaMemcpy(h.data(), dRes, N * sizeof(int), cudaMemcpyDeviceToHost)); // копируем результат GPU на CPU

    CUDA_CHECK(cudaFree(dA));                                         // освобождаем память dA на GPU
    CUDA_CHECK(cudaFree(dB));                                         // освобождаем память dB на GPU
}

// Main: генерация данных и сравнение CPU/GPU

int main() {
    vector<int> sizes = {10000, 100000};                              // размеры массивов, которые требуются в задании

    mt19937 gen(random_device{}());                                   // генератор случайных чисел
    uniform_int_distribution<int> dist(0, 100000);                    // распределение: числа от 0 до 100000

    for (int N : sizes) {                                             // прогоняем эксперимент для каждого N
        cout << "Размер массива N = " << N << "\n";                   // печатаем текущий размер

        vector<int> a(N);                                             // создаём исходный массив на CPU
        for (int i = 0; i < N; ++i) a[i] = dist(gen);                 // заполняем массив случайными значениями

        vector<int> cpu = a, gpu = a;                                 // делаем две копии: одна пойдёт в CPU sort, другая - в GPU sort

        auto c1 = chrono::high_resolution_clock::now();               // начало замера CPU времени
        sort(cpu.begin(), cpu.end());                                 // сортировка на CPU (эталон / сравнение)
        auto c2 = chrono::high_resolution_clock::now();               // конец замера CPU времени
        double cpu_ms = chrono::duration<double, milli>(c2 - c1).count(); // переводим длительность в миллисекунды

        cudaEvent_t e1, e2;                                           // CUDA events - более корректный способ мерить время на GPU
        CUDA_CHECK(cudaEventCreate(&e1));                              // создаём событие начала
        CUDA_CHECK(cudaEventCreate(&e2));                              // создаём событие конца

        CUDA_CHECK(cudaEventRecord(e1));                               // ставим “метку времени” на GPU перед запуском сортировки
        gpuMergeSort(gpu);                                            // выполняем сортировку на GPU (внутри: kernels + копирования)
        CUDA_CHECK(cudaEventRecord(e2));                               // ставим “метку времени” на GPU после сортировки
        CUDA_CHECK(cudaEventSynchronize(e2));                          // ждём, пока событие e2 реально завершится (значит GPU закончил работу)

        float gpu_ms = 0.0f;                                          // сюда запишем время GPU в миллисекундах
        CUDA_CHECK(cudaEventElapsedTime(&gpu_ms, e1, e2));             // вычисляем разницу между e1 и e2 (это и есть GPU time)

        CUDA_CHECK(cudaEventDestroy(e1));                              // удаляем событие начала (чтобы не текли ресурсы)
        CUDA_CHECK(cudaEventDestroy(e2));                              // удаляем событие конца

        cout << "CPU time: " << cpu_ms << " ms\n";                    // выводим CPU время
        cout << "GPU time: " << gpu_ms << " ms\n";                    // выводим GPU время

        if (gpu_ms > 0.0f)                                            // проверка на ноль (чтобы не делить на 0)
            cout << "Speedup (CPU/GPU): " << (cpu_ms / gpu_ms) << "x\n"; // ускорение: >1 значит GPU быстрее, <1 значит CPU быстрее
    }

    return 0;                                                         // корректное завершение программы
}


Overwriting gpu_merge_sort.cu


In [10]:
!nvcc -O3 -arch=sm_75 gpu_merge_sort.cu -o gpu_merge_sort
!./gpu_merge_sort

Размер массива N = 10000
CPU time: 0.552883 ms
GPU time: 0.600704 ms
Speedup (CPU/GPU): 0.920392x
Размер массива N = 100000
CPU time: 6.88141 ms
GPU time: 0.751328 ms
Speedup (CPU/GPU): 9.15899x


##**Вывод**

В ходе решения задачи 4 была измерена производительность сортировки на CPU и GPU для массивов размером 10 000 и 100 000 элементов.

Для массива из 10 000 элементов время выполнения на CPU составило 0.55 мс, а на GPU 0.60 мс. Ускорение оказалось меньше единицы (0.92x), что означает, что при таком размере данных использование GPU не даёт выигрыша по времени. Это объясняется накладными расходами на передачу данных между CPU и GPU, а также на запуск вычислительных ядер, которые становятся значимыми при малых объёмах данных.

Для массива из 100 000 элементов время выполнения на CPU составило 6.88 мс, тогда как на GPU 0.75 мс. В этом случае было получено ускорение около 9.16x, что демонстрирует высокую эффективность параллельной обработки на GPU при достаточно большом объёме данных.

Таким образом, результаты показывают, что использование GPU оправдано и эффективно для обработки больших массивов, тогда как для малых массивов последовательная реализация на CPU может быть более выгодной из-за меньших накладных расходов.